In [25]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import LineString, Polygon
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys
import os
from PIL import Image
import matplotlib.pyplot as plt
import tifffile as tiff
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]
SRC_DIR = PROJECT_ROOT / "src" / "spacenet"
sys.path.append(str(SRC_DIR))
from dataset.data_processing import get_shapes, rasterize_lines

In [2]:
data_dir = PROJECT_ROOT / "data"
data_dir.exists() or sys.exit(f"Data directory {data_dir} does not exist. Please check the path.")

True

In [11]:
location = "Germany"
split = "Training"
data_types = ["label_image_mapping", "reference"]
naming_dict = {'label_image_mapping': 'scenes', 'reference': 'objects'}
dfs = {naming_dict[data_type]: pd.read_csv(data_dir / f"{location}_{split}" / f"{location}_{split}_Public_{data_type}.csv") for data_type in data_types}

In [20]:
image_id = '10500500C4DD7000_0_18_69'
print((dfs['objects']['ImageId']==image_id).sum())

2


In [21]:
dfs['objects'][dfs['objects']['ImageId']==image_id]

,ImageId,Object,Wkt_Pix,Flooded,length_m,travel_time_s
3910,10500500C4DD7000_0_18_69,Road,LINESTRING EMPTY,Null,Null,Null
8163,10500500C4DD7000_0_18_69,Building,"POLYGON ((763.578600214536 240.12157359426553,...",False,Null,Null


In [26]:
lines, _ = get_shapes(dfs['objects'], image_id)
if lines.iloc[0] == LineString():
    print('yes')

yes


In [28]:
dfs['scenes']

,label,pre-event image,post-event image 1,post-event image 2
0,0_41_59.geojson,10500500C4DD7000_0_41_59.tif,10500500E6DD3C00_0_41_59.tif,NaN
1,0_45_64.geojson,10500500C4DD7000_0_45_64.tif,10500500E6DD3C00_0_45_64.tif,NaN
2,0_32_59.geojson,10500500C4DD7000_0_32_59.tif,10500500E6DD3C00_0_32_59.tif,NaN
3,0_38_61.geojson,10500500C4DD7000_0_38_61.tif,10500500E6DD3C00_0_38_61.tif,NaN
4,0_36_60.geojson,10500500C4DD7000_0_36_60.tif,10500500E6DD3C00_0_36_60.tif,NaN
...,...,...,...,...
197,0_42_62.geojson,10500500C4DD7000_0_42_62.tif,10500500E6DD3C00_0_42_62.tif,NaN
198,0_31_62.geojson,10500500C4DD7000_0_31_62.tif,10500500E6DD3C00_0_31_62.tif,NaN
199,0_31_66.geojson,10500500C4DD7000_0_31_66.tif,10500500E6DD3C00_0_31_66.tif,NaN
200,0_27_62.geojson,10500500C4DD7000_0_27_62.tif,1040050035DC3B00_0_27_62.tif,10500500E6DD3C00_0_27_62.tif


In [32]:
col = 'pre-event image'
label = '0_18_69.geojson'
row = dfs['scenes'][dfs['scenes']['label']==label]
img = Image.open(data_dir / f"{location}_{split}" / ('PRE-event' if col=='pre-event image' else 'POST-event') / row[col].values[0])
img_array = np.array(img.convert('RGB'))

In [33]:
print('min: ', np.min(img_array), 'max: ', np.max(img_array))

min:  0 max:  243


In [34]:
img_array.shape

(1300, 1300, 3)